In [5]:
import numpy as np

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
from sklearn.metrics import confusion_matrix

In [8]:
states = np.load('states2.npy')
hands = np.load('hands.npy')

In [10]:
states2d = states.reshape(2571176, 4*52)

In [11]:
states.shape

(2571176, 4, 52)

In [28]:
sum_states2d = np.sum(states2d, axis=1)

In [31]:
sum_states2d[:50]

array([12., 12., 14., 14., 16., 16., 18., 18., 20., 20., 22., 22., 24.,
       12., 12., 14., 14., 16., 16., 18., 18., 20., 20., 22., 22., 24.,
       12., 12., 14., 14., 16., 16., 18., 18., 20., 20., 22., 22., 12.,
       12., 14., 14., 16., 16., 18., 18., 20., 20., 22., 22.])

In [36]:
tempX = states2d[sum_states2d>20]

In [37]:
tempY = hands[sum_states2d>20]

In [38]:
tempY.shape

(737099, 52)

In [4]:
states[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [62]:
rf0 = RandomForestClassifier()

In [63]:
rf0.fit(states2d[:100000], hands[:100000])

RandomForestClassifier()

In [16]:
def predict_hand(probs, t):
    return np.array([roundt(a[0][1], t) for a in probs])

In [90]:
rf0.predict_proba(states2d[100000:100001])[:3]

[array([[0.97, 0.03]]), array([[0.25, 0.75]]), array([[0.2, 0.8]])]

In [99]:
np.array(rf0.predict_proba(states2d[100000:100002]))[:,:,1][:,0]

array([0.03, 0.75, 0.8 , 0.09, 0.09, 0.27, 0.04, 0.06, 0.07, 0.03, 0.04,
       0.03, 0.04, 0.8 , 0.58, 0.09, 0.78, 0.1 , 0.02, 0.08, 0.8 , 0.77,
       0.73, 0.02, 0.04, 0.05, 0.08, 0.05, 0.1 , 0.76, 0.1 , 0.08, 0.03,
       0.03, 0.06, 0.01, 0.05, 0.05, 0.02, 0.12, 0.78, 0.03, 0.04, 0.04,
       0.08, 0.02, 0.11, 0.07, 0.03, 0.02, 0.01, 0.03])

In [88]:
rf0.predict_proba(states2d[100000:100002])

[array([[0.97, 0.03],
        [0.23, 0.77]]), array([[0.25, 0.75],
        [0.96, 0.04]]), array([[0.2 , 0.8 ],
        [0.98, 0.02]]), array([[0.91, 0.09],
        [0.89, 0.11]]), array([[0.91, 0.09],
        [0.92, 0.08]]), array([[0.73, 0.27],
        [0.9 , 0.1 ]]), array([[0.96, 0.04],
        [0.26, 0.74]]), array([[0.94, 0.06],
        [0.97, 0.03]]), array([[0.93, 0.07],
        [0.95, 0.05]]), array([[0.97, 0.03],
        [0.97, 0.03]]), array([[0.96, 0.04],
        [0.95, 0.05]]), array([[0.97, 0.03],
        [0.98, 0.02]]), array([[0.96, 0.04],
        [0.98, 0.02]]), array([[0.2 , 0.8 ],
        [0.97, 0.03]]), array([[0.42, 0.58],
        [0.95, 0.05]]), array([[0.91, 0.09],
        [0.88, 0.12]]), array([[0.22, 0.78],
        [0.97, 0.03]]), array([[0.9 , 0.1 ],
        [0.93, 0.07]]), array([[0.98, 0.02],
        [0.88, 0.12]]), array([[0.92, 0.08],
        [0.24, 0.76]]), array([[0.2 , 0.8 ],
        [0.96, 0.04]]), array([[0.23, 0.77],
        [0.97, 0.03]]), array([[0

In [71]:
predict_hand(rf0.predict_proba(states2d[100001:100002]), 0.3)

array([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0])

In [84]:
hands[1000000:1000001]-predict_hand(rf0.predict_proba(states2d[1000000:1000001]), 0.3)

array([[ 1.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -1.,  0.,  0.,  1.,  1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,
        -1.,  1., -1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1., -1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [19]:
def roundt(x, t):
    return 1 if x > t else 0

In [49]:
np.array([roundt(a[0][1], 0.3) for a in rf0.predict_proba(tempX[10000:10001])])

array([0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0])

In [52]:
confusion_matrix(tempY[10000], np.array([roundt(a[0][1], 0.3) for a in rf0.predict_proba(tempX[10000:10001])]))

array([[33,  9],
       [ 5,  5]])

In [55]:
confusion_matrix(tempY[10001], np.array([roundt(a[0][1], 0.3) for a in rf0.predict_proba(tempX[10001:10002])]))

array([[37,  5],
       [ 3,  7]])

In [57]:
confusion_matrix(tempY[10002], np.array([roundt(a[0][1], 0.25) for a in rf0.predict_proba(tempX[10002:10003])]))

array([[31, 11],
       [ 2,  8]])

In [12]:
def get_metrics(model, X, Y, t):
    all_probs = model.predict_proba(X)
    accs, precs, recs = [], [], []
    for i,y in enumerate(Y):
        probs = np.array(all_probs)[:,:,1][:,i]
        preds = [roundt(p, t) for p in probs]
        tn, fp, fn, tp = confusion_matrix(y, preds).ravel()
        acc, prec, rec = (tp+tn)/(tn+fp+fn+tp), tp/(tp+fp), tp/(tp+fn)
        accs.append(acc)
        precs.append(prec)
        recs.append(rec)
    return accs, precs, recs

In [114]:
accs, precs, recs = get_metrics(rf0, states22d[100000:101000], hands[100000:101000], 0.3)

In [115]:
np.mean(accs), np.mean(precs), np.mean(recs)

(0.7914038461538462, 0.4351283355533355, 0.2911)

In [116]:
accs, precs, recs = get_metrics(rf0, states22d[100000:101000], hands[100000:101000], 0.25)

In [117]:
np.mean(accs), np.mean(precs), np.mean(recs)

(0.7448076923076924, 0.38473894521800867, 0.5337000000000001)

# save

In [1]:
import dill

In [124]:
with open("rf.obj","wb") as f:
    dill.dump(rf0,f)

In [2]:
model = dill.load(open("rf.obj","rb"))

In [3]:
model

RandomForestClassifier()

In [13]:
# state is array of length 208 made from flattened 4x52 array where 
# row 0 is past discards (cards opponent discarded)
# row 1 is known cards in opponent's hand (picked up by opponent and not discarded)
# row 2 is past faceUpCards not picked up by opponent
# row 3 is your own hand
def predict_opponent_hand(rf, state):
    probs = rf.predict_proba([state])
    return np.array(probs)[:,:,1][:,0]

In [25]:
def predict_cards_t(hand_probs, t):
    return np.array([roundt(p, t) for p in hand_probs])

In [29]:
def predict_cards_max(hand_probs):
    inds = hand_probs.argsort()[-10:][::-1]
    ret = np.zeros((52))
    ret[inds] = 1
    return ret

In [22]:
states[100000].shape

(4, 52)

In [14]:
predict_opponent_hand(model, states2d[100000])

array([0.03, 0.75, 0.8 , 0.09, 0.09, 0.27, 0.04, 0.06, 0.07, 0.03, 0.04,
       0.03, 0.04, 0.8 , 0.58, 0.09, 0.78, 0.1 , 0.02, 0.08, 0.8 , 0.77,
       0.73, 0.02, 0.04, 0.05, 0.08, 0.05, 0.1 , 0.76, 0.1 , 0.08, 0.03,
       0.03, 0.06, 0.01, 0.05, 0.05, 0.02, 0.12, 0.78, 0.03, 0.04, 0.04,
       0.08, 0.02, 0.11, 0.07, 0.03, 0.02, 0.01, 0.03])

In [26]:
predict_cards_t(predict_opponent_hand(model, states2d[100000]), 0.3)

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [30]:
predict_cards_max(predict_opponent_hand(model, states2d[100000]))

array([0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1.,
       0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])